# Title: Python Running Environment Discoverer 
## Author: Steven K Armour 
(github: https://github.com/GProtoZeroW); twitter:https://twitter.com/ArmourMSG; Mastadon: https://mastodon.online/@ArmourMSG

# Motivation

So between Python versions, virtual environments, virtual machines \[Window's Windows Subsystem for Linux (WSL) is a virtual machine built upon Windows Hyper-V (https://learn.microsoft.com/en-us/windows/wsl/faq \], and dockers the question comes where in the hell is my code running? Now, if you have set up everything yourself, answering where your code is running should be straightforward. But as soon as IDEs start doing some level of automated environment setup, not to mention docker, this can quickly become unknown. So then let's instead have Python tell us itself, and that is what the following script tries to do. (note: as of 20230218, I have not tested the docker part)

# Imports

In [1]:
#%%writefile py_running_environment_discoverer.py

import os
import platform
import sys
from pathlib import Path

In [2]:
#%%writefile -a py_running_environment_discoverer.py

########################
# Environment Inspection
########################

#getters
# Useful when working with PyCharm, Windows, WSL, and Conda

def get_os():
    system = platform.system()
    if system == 'Linux':
        with open('/proc/version', 'r') as f:
            if 'microsoft' in f.read().lower():
                return 'Windows Subsystem for Linux (WSL)'
        return 'Linux'
    elif system == 'Darwin':
        return 'macOS'
    elif system == 'Windows':
        return 'Windows'
    else:
        return 'Unknown OS'

def in_virtual_env():
    return hasattr(sys, 'real_prefix') or (hasattr(sys, 'base_prefix') and sys.base_prefix != sys.prefix)

def get_virtual_env_info():
    if in_virtual_env():
        venv_path = sys.prefix
        return {
            "is_virtual_env": True,
            "path": venv_path,
            "name": os.path.basename(venv_path),
            "is_conda": os.path.exists(os.path.join(venv_path, 'conda-meta'))
        }
    return {"is_virtual_env": False}

def in_docker():
    path = '/proc/self/cgroup'
    return os.path.exists('/.dockerenv') or os.path.isfile(path) and any('docker' in line for line in open(path))

def get_docker_container_id():
    if in_docker():
        with open('/proc/self/cgroup', 'r') as f:
            for line in f:
                if 'docker' in line:
                    return line.split('/')[-1].strip()
    return None

def get_current_path():
    return os.getcwd()



# Running the above

If I ran the above cells (with all of them set to be code cells) for me in this notebook with the PyCharm setup I have for the Blogs project yields: (Where I have omitted parts of the output for privacy)

```
Operating System: Windows Subsystem for Linux (WSL)
Current Path: ~~/UC_PyEng_Posts/IDE_Tips/Environment_Discovery
Python Version: 3.11.7 (main, Dec  8 2023, 18:56:57) [GCC 9.4.0]
Running in Virtual Environment: Yes
Virtual Environment Path: ~~/.virtualenvs/UC_PyEng_Posts
Virtual Environment Name: UC_PyEng_Posts
Virtual Environment Type: Python
To activate this environment, run: `source ~~/.virtualenvs/UC_PyEng_Posts/bin/activate`
Running in Docker: No
```

# Exstracting the code from this Development Notebook to a Python File

So then to get the above code out into a script, we will use an earlier post about extracting code from notebooks by parsing cells for `#%%writefile` (Better_Jupyter_Writefile_Extractor_Tool_DevNB)[https://github.com/GProtoZeroW/UC_PyEng_Posts/blob/main/Jupyter_Tips/Better_Writefile_Extractor_Tool/Better_Jupyter_Writefile_Extractor_Tool_DevNB.ipynb]

In [3]:
import sys

# Get the current working directory (cwd) of the notebook
cwd = Path.cwd()

# Construct the path to the script directory
script_dir = cwd.parents[1] / 'Jupyter_Tips/Better_Writefile_Extractor_Tool'

# Append the script directory to sys.path
sys.path.append(str(script_dir))

from jupyter_writefile_extractor import Notebook_Writefile_Extractor


In [4]:
exstract_myself=Notebook_Writefile_Extractor(Path.cwd()/'Python_Running_Environment_Discoverer_DevNB.ipynb')
exstract_myself.write_to_files()